## 10. Bash Scripting

As systems admins in the IT industry using Linux systems, there are many tasks that are performed repetitively (regular patching or backups) -> *Robotic Tasks*

Instead of doing that every time, you put all the commands in a text file and let the system execute the file for you (bash script). -> The word Bash is for Bash Shell

* Kinds of shells:
    
    - Bourne shell (`sh`): the first default shell on Unix systems, released in 1979

    - C shell (`csh`): Linux shell from the late 1970s whose main objective was to improve interactive use and mimic the C language

    - Tenex C shell (`tcsh`): extension of the C shell (`csh`) merged in the early 1980s

    - KornShell (`ksh`): Unix shell and language based on the Bourne shell (`sh`) developed in the early 1980s

    - Debian Almquist shell (`dash`): Unix shell developed in the late 1990s from the Almquist shell (ash), which was ported to Debian and renamed. Dash is famous for being the default shell for Ubuntu and Debian

    - Bourne Again shell (`bash`): Unix shell and command language created as an extension of the Bourne shell (sh) in 1989. The shell program is the default login shell for many Linux distributions and earlier versions of macOS.

### Bash Scripting Practice

Let's do the scripting practice using the following `Vagrantfile`:

```
Vagrant.configure("2") do |config|

  config.vm.define "scriptbox" do |scriptbox|
    scriptbox.vm.box = "geerlingguy/centos7"
	scriptbox.vm.network "private_network", ip: "192.168.10.12"
        scriptbox.vm.hostname = "scriptbox"
	scriptbox.vm.provider "virtualbox" do |vb|
     vb.memory = "1024"
   end
  end

  config.vm.define "web01" do |web01|
    web01.vm.box = "geerlingguy/centos7"
	web01.vm.network "private_network", ip: "192.168.10.13"
        web01.vm.hostname = "web01"
  end
  
  config.vm.define "web02" do |web02|
    web02.vm.box = "geerlingguy/centos7"
	web02.vm.network "private_network", ip: "192.168.10.14"
        web02.vm.hostname = "web02"
  end

   config.vm.define "web03" do |web03|
    web03.vm.box = "ubuntu/bionic64"
        web03.vm.network "private_network", ip: "192.168.10.15"
        web03.vm.hostname = "web03"
  end
end
```

* Bring up `scriptbox` VM and create a directory in `/opt/scripts`

* A ***script*** is a text file that contains "bash" commands. Use `vim` to create a script.

    `vim firstscript.sh`

* First line is `#!/bin/bash` (path of the interpreter) -> "Open this interpreter and execute all the commands in this script on that interpreter"

    - If it is a python/ruby script, you'll have the path of a Python/Ruby interpreter
    
    - Since it is a writing bash script, you have the bash interpreter

* Use `echo` to print messages

* Use `#` for comments

```
#!/bin/bash

echo "Welcome to bash script"
echo

echo "The uptime of the system is:"
uptime

echo "Memory Utilization:"
free -m

echo "Disk Utilization:"
df -h
```

* To run a script, give its absolute or relative path: `./firstscript.sh`

* Modify permission if necessary: 

    - Check execution permissions (4th element): `ls -l` 

    - Give execution permission: `chmod +x firstscript.sh`

    - Now you can execute the script


### Websetup script

* Choose a webpage template from Tooplate, e.g.: https://www.tooplate.com/view/2098-health

* Create a script file using vim: `vim /opt/scripts/websetup.sh`

* Add the commands with the steps for the web setup:

```
  1 #!/bin/bash
  2
  3 # Installing dependencies (> to send output to null - do not show)
  4 echo "#####################################"
  5 echo "Installing packages"
  6 echo "#####################################"
  7 sudo yum install wget unzip httpd -y > /dev/null
  8 echo
  9
 10 # Start and enable services
 11 echo "#####################################"
 12 echo "Start & enable HTTPD Service"
 13 echo "#####################################"
 14 sudo systemctl start httpd
 15 sudo systemctl enable httpd
 16 echo
 17
 18 # Creating a temporary directory
 19 echo "#####################################"
 20 echo "Starting artifact deployment"
 21 echo "#####################################"
 22 mkdir -p /tmp/webfiles
 23 cd /tmp/webfiles
 24 echo
 25
 26 # Get and copy webpage template to the /var/www/html directory
 27 wget https://www.tooplate.com/zip-templates/2098_health.zip > /dev/null
 28 unzip 2098_health.zip > /dev/null
 29 sudo cp -r 2098_health/* /var/www/html/
 30 echo
 31
 32 # Bounce service
 33 echo "#####################################"
 34 echo "Restarting HTTPD Service"
 35 echo "#####################################"
 36 systemctl restart httpd
 37 echo
 38
 39 # Clean up temporary files
 40 echo "#####################################"
 41 echo "Removing temporary files"
 42 echo "#####################################"
 43 rm -rf /tmp/webfiles
 44
 45 # Print status of the service
 46 sudo systemctl status httpd
 47
 48 # Show the content of the /var/www/html directory
 49 ls /var/www/html/
```

* Give execution permission to the script file: `chmod +x /opt/scripts/websetup.sh`

* Execute the script file: `/opt/scripts/websetup.sh`

* Change any errors or prevent unnecessary outputs from showing

* Execute again and check the webpage status using the VM's IP address: `ifconfig`

* Open the VM's IP address in a browser.

### Variables

Temporary stores of information in memory. There are two actions we may perform for variables:

* Setting a value for a variable

* Reading or using the value for a variable

To define a variable:

`VariableName=Value`

To retrieve a variable:

`$VariableName`

To print a variable:

`echo $VariableName`

### Use Variables in Scripting

* An enhanced websetup script using variables:

* Define variables at the beginning of the file:

```
  3 # Variables
  4 PACKAGE="httpd wget unzip"
  5 SVC="httpd"
  6 URL='https://www.tooplate.com/zip-templates/2098_health.zip'
  7 ART_NAME='2098_health'
  8 TEMPDIR="/tmp/webfiles"
```

$\rightarrow$ To select variables, choose those that change or vary based on different environments, projects or use cases.

* Final script with variables:

```
  1 #!/bin/bash
  2
  3 # Variables
  4 PACKAGE="httpd wget unzip"
  5 SVC="httpd"
  6 URL='https://www.tooplate.com/zip-templates/2098_health.zip'
  7 ART_NAME='2098_health'
  8 TEMPDIR="/tmp/webfiles"
  9
 10 # Installing dependencies (> to send output to null - do not show)
 11 echo "#####################################"
 12 echo "Installing packages"
 13 echo "#####################################"
 14 sudo yum install $PACKAGE -y > /dev/null
 15 echo
 16
 17 # Start and enable services
 18 echo "#####################################"
 19 echo "Start & enable HTTPD Service"
 20 echo "#####################################"
 21 sudo systemctl start $SVC
 22 sudo systemctl enable $SVC
 23 echo
 24
 25 # Creating a temporary directory
 26 echo "#####################################"
 27 echo "Starting artifact deployment"
 28 echo "#####################################"
 29 mkdir -p $TEMPDIR
 30 cd $TEMPDIR
 31 echo
 32
 33 # Get and copy webpage template to the /var/www/html directory
 34 wget $URL > /dev/null
 35 unzip $ART_NAME.zip > /dev/null
 36 sudo cp -r $ART_NAME/* /var/www/html/
 37 echo
 38
 39 # Bounce service
 40 echo "#####################################"
 41 echo "Restarting HTTPD Service"
 42 echo "#####################################"
 43 systemctl restart $SVC
 44 echo
 45
 46 # Clean up temporary files
 47 echo "#####################################"
 48 echo "Removing temporary files"
 49 echo "#####################################"
 50 rm -rf $TEMPDIR
 51
 52 # Print status of the service
 53 sudo systemctl status $SVC
 54
 55 # Show the content of the /var/www/html directory
 56 ls /var/www/html/
```

### Command Line Arguments

* You can use number-variables from `$0` to `$9` to set variables in the command line as arguments.

* For example, the following script named `4_args.sh`:

```
#!/bin/bash

echo "Value of 0 is:"
echo $0

echo "Value of 1:"
echo $1

echo "Value of 2:"
echo $2

echo "Value of 3:"
echo $3
```

* Running the script results in the following lines:

```
Value of 0 is:
./4_args.sh
Value of 1:

Value of 2:

Value of 3:

```

* `$0` is always the name of the script. From `$1` to `$9` you can add arguments in the command line to assign values to these variables:

`[root@scriptbox scripts]# ./4_args.sh Linux AWS Ansible Jankins`

```
Value of 0 is:
./4_args.sh
Value of 1:
Linux
Value of 2:
AWS
Value of 3:
Ansible
```

* The values of Linux, AWS and Ansible are assigned to variables `$1`, `$2` and `$3`, respectively.

### Use Command Line Arguments in Scripting

* Now, we want to add the URL and artifact name as arguments in the command line.

* We comment the variable definition for `$URL` and `$ART_NAME` in the script (lines 6 and 7) and replace the variables for `$1` and `$2`, respectively in lines 34-36.

* The new script is as follows:

```
  1 #!/bin/bash
  2
  3 # Variables
  4 PACKAGE="httpd wget unzip"
  5 SVC="httpd"
  6 # URL='https://www.tooplate.com/zip-templates/2098_health.zip' (commented vars)
  7 # ART_NAME='2098_health'
  8 TEMPDIR="/tmp/webfiles"
  9
 10 # Installing dependencies (> to send output to null - do not show)
 11 echo "#####################################"
 12 echo "Installing packages"
 13 echo "#####################################"
 14 sudo yum install $PACKAGE -y > /dev/null
 15 echo
 16
 17 # Start and enable services
 18 echo "#####################################"
 19 echo "Start & enable HTTPD Service"
 20 echo "#####################################"
 21 sudo systemctl start $SVC
 22 sudo systemctl enable $SVC
 23 echo
 24
 25 # Creating a temporary directory
 26 echo "#####################################"
 27 echo "Starting artifact deployment"
 28 echo "#####################################"
 29 mkdir -p $TEMPDIR
 30 cd $TEMPDIR
 31 echo
 32
 33 # Get and copy webpage template to the /var/www/html directory
 34 wget $1 > /dev/null
 35 unzip $2.zip > /dev/null
 36 sudo cp -r $2/* /var/www/html/
 37 echo
 38
 39 # Bounce service
 40 echo "#####################################"
 41 echo "Restarting HTTPD Service"
 42 echo "#####################################"
 43 systemctl restart $SVC
 44 echo
 45
 46 # Clean up temporary files
 47 echo "#####################################"
 48 echo "Removing temporary files"
 49 echo "#####################################"
 50 rm -rf $TEMPDIR
 51
 52 # Print status of the service
 53 sudo systemctl status $SVC
 54
 55 # Show the content of the /var/www/html directory
 56 ls /var/www/html/
```

* Now, the user needs to pass two arguments: the URL and the Artifact name. Let's change the webpage template:

`[root@scriptbox scripts]# ./5_args_websetup.sh https://www.tooplate.com/zip-templates/2091_ziggy.zip 2091_ziggy`

* In case the user does not pass the arguments, an error will occur when running the script: `$1` and `$2` will be empty.

* Now the script code can be reusable to deploy any website we want.

### System Variables

* `$0` - The name of the bash script

* `$1 - $9` - The first 9 arguments to the bash script

* `$#` - How many arguments were passed to the bash script

* `$@` - All the arguments supplied to the bash script

* `$?` - The exit status of the most recently run process 

    - `0` means the last command was successful
    
    - a nonzero value means that the last command failed

* `$$` - The process ID of the current script

* `$USER` - The username of the user running the script

* `$HOSTNAME` - The hostname of the machine the script is running on

* `$SECONDS` - The number of seconds since the script was started

* `$RANDOM` - Returns a different random number each time it is referred to

* `$LINENO` - Returns the current line number in the bash script

### Quotes

Two types of quotes: `""` and `''`.

Quotes are very useful to store sentences and special characters

* When working with special characters, use `\` to remove the special meaning of the next character: `\$`

* `""` keeps the meaning of variables, whereas `''` does not.

### Command Substitution

Command substitution takes the output of a command and stores it into a variable.

You can achieve that by using `` or `$()`

For example: Save the output of `uptime` into a variable

`UP=$(uptime)`

```
# echo $UP
14:49:37 up 1:19, 1 user, load average: 0.00, 0.01, 0.05
```

`CURRENT_USER=$(who)`

```
# echo $CURRENT_USER
vagrant pts/0 2023-09-20 10:46 (10.0.2.2) vagrant pts/1 2023-09-20 10:47 (10.0.2.2)
```

Choose the current free ram (765) from the command `free -m`:

```
# free -m
              total        used        free      shared  buff/cache   available
Mem:            990         113         765           6         111         748
Swap:          1023           0        1023
```

We can get that piece of information by using:

```
# free -m | grep Mem | awk '{print $4}'
765
```

Then, we can save it in a new variable `FREE_RAM`:

```
# FREE_RAM=`free -m | grep Mem |awk '{print $4}'`
```

We can then print (or use) this information (however we want):

```
# echo "Free RAM is $FREE_RAM mb"
Free RAM is 764 mb
```

### Using Command Substitution in Scripting

Check the following example of command substitution for scripting:

```
  1 #!/bin/bash
  2
  3 echo "Welcome $USER on $HOSTNAME"
  4 echo "###########################################"
  5
  6 FREERAM=$(free -m | grep Mem | awk '{print $4}')
  7 LOAD=`uptime | awk '{print $9}'`
  8 ROOTFREE=$(df -h | grep '/dev/sdal' | awk '{print $4}')
  9
 10 echo "###########################################"
 11 echo "Available free RAM is $FREERAM MB"
 12 echo "###########################################"
 13 echo "Current Load Average is $LOAD"
 14 echo "###########################################"
 15 echo "Free ROOT partition size is $ROOTFREE"
```

### Exporting Variables

It helps to keep permanent variables for a user or for every user in the system.

If you define the following variable:

`# SEASON="Monsoon"`
```
# echo $SEASON
Monsoon
```

The problem is that everytime you log out (close the current shell), all the variables will be erased (local/temporary variables). 

* Exporting a variable, makes the variable global for all other "child" shells (still temporary):

`# export SEASON`

* To make a variable permanent for a user, you need to place the export command and variable in a hidden file `.bashrc` located in every user's home directory:

`# ls -a`

```
.   anaconda-ks.cfg  .bash_logout   .bashrc  original-ks.cfg  .tcshrc
..  .bash_history    .bash_profile  .cshrc   .pki             .viminfo
```

* Run to edit the `.bashrc` file (see line 14):

`# vi .bashrc`

```
      1 # .bashrc
      2
      3 # User specific aliases and functions
      4
      5 alias rm='rm -i'
      6 alias cp='cp -i'
      7 alias mv='mv -i'
      8
      9 # Source global definitions
     10 if [ -f /etc/bashrc ]; then
     11         . /etc/bashrc
     12 fi
     13
     14 export SEASON="Monsoon"
```

* After this, you need to log out and back in again so the command `# source .bashrc` runs. After logging back in, the variable `SEASON` will be permanent for that user even after logging out.

* If you want to make a variable permanent globally for **every user**, you need to add the export command in the file `/etc/profile` (add same line 14 at the end or with other value):

`# vim /etc/profile`

* If you log out and log back in, you can see that the variable `$SEASON` keeps its value.

* When modifying both `/etc/profile` and `~/.bashrc` of an user, the value of the variable exported in the `.bashrc` file of that user overrides the one in the `/etc/profile` file.

### User Input

We would like to take input from the user while executing the script, storing it into a variable and then using that variable in the script.

Examples of inputs from user:

* IP addresses
* Usernames
* Passwords
* Confirmation Y/N

For taking user inputs, use the command `read varname`.

### Using `read` Command in Scripting

* Let's create a new script: `7_user_input.sh`:

```
  1 #!/bin/bash
  2
  3 echo "Enter your skills:"
  4 read SKILL
  5
  6 echo "Your $SKILL skill is in high demand in the IT Industry"
  7
  8 read -p 'Username: ' USR
  9 read -sp 'Password: ' pass
 10
 11 echo
 12
 13 echo "Login Successful: Welcome USER $USR"
```

* `-p` in line 8 is for "prompt" option

* `-sp` option in line 9 is for hiding the typing (mostly used for passwords)

* For the case of DevOps tasks, it is not recommended to take user input to make the scripts interactive, because we run scripts from background from some other tools and we don't want users to interact with the system.

### Conditions: IF Statement

Syntax:

```
if [<some test>]
then
  <commands>
else
  <other commands>
fi
```

For example:

```
  1 #!/bin/bash
  2
  3 read -p "Enter a number: " NUM
  4 echo
  5
  6 if [ $NUM -gt 100 ]
  7 then
  8    echo "We have entered the IF block"
  9    sleep 3
 10    echo "Your number is greater than 100"
 11    echo
 12    date
 13 else
 14    echo "Your number is less than 100"
 15    echo
 16 fi
 17
 18 echo "Script execution completed successfully"
```

* `-gt` = greater than

#### `elif` Condition: 

* We want a script to get the IP addresses that are working on the VM (omitting the LOOPBACK address). We will use the keyword `mtu` which is repeated in every IP showing in the command `ip addr show`. So we will count the number of occurences of `mtu` ignoring the first one (LOOPBACK):

`# ip addr show | grep -v LOOPBACK | grep -ic mtu`

`2`

* `-ic`: ignores the case and counts the number of that occurrence

```
  1 #!/bin/bash
  2
  3 value=$(ip addr show | grep -v LOOPBACK | grep -ic mtu)
  4
  5 if [ $value -eq 1 ]
  6 then
  7    echo "1 Active network interface found"
  8 elif [ $value -gt 1 ]
  9 then
 10    echo "Multiple active interfaces found"
 11 else
 12    echo "No active interfaces found"
 13 fi
```

* `-eq`: equal to

### Bash Operators

`! EXPRESSION` = The EXPRESSION is false

`-n STRING` = The length of STRING is greater than zero

`-z STRING` = The length of STRING is zero (i.e., it is empty)

`STRING1 = STRING2` = STRING1 is equal to STRING2

`STRING1 != STRING2` = STRING1 is not equal to STRING2

`INTEGER1 -eq INTEGER2` = INTEGER1 is numerically equal to INTEGER2

`INTEGER1 -gt INTEGER2` = INTEGER1 is numerically greater than INTEGER2

`INTEGER1 -lt INTEGER2` = INTEGER1 is numerically less than INTEGER2

`-d FILE` = FILE (path) exists and is a directory

`-e FILE` = FILE exists

`-r FILE` = FILE exists and the read permission is granted

`s FILE` = FILE exists and its size is greater than zero (i.e., it's not empty)

`-w FILE` = FILE exists and the write permission is granted

`-x FILE` = FILE exists and the execute permission is granted

* The following script detects if a process is running or if it's not running, it will start the process:

* Remember that every process that is running has a `.pid` file in `/var/run/processname`. For example:

    - When `httpd` process is running: 

        `# cat /var/run/httpd/httpd.pid`

        `3136`

    - When it is not running:

         `# cat /var/run/httpd/httpd.pid`

        `cat: /var/run/httpd/httpd.pid: No such file or directory`

* You can also check if a process is running by calling `echo $?` after running `cat /var/run/httpd/httpd.pid`. If it is `0` (the `.pid` file exists), then the process is running, else, it is not running.

```
  1 #!/bin/bash
  2
  3 echo "##############################################"
  4 date
  5 ls /var/run/httpd/httpd.pid &> /dev/null
  6
  7 if [ $? -eq 0 ]
  8 then
  9    echo "httpd process is running"
 10 else
 11    echo "httpd process is NOT running"
 12    echo "Starting the httpd process"
 13    systemctl start httpd
 14
 15    if [ $? -eq 0 ]
 16    then
 17       echo "Process started successfully"
 18    else
 19       echo "Process starting FAILED. Contact support"
 20    fi
 21 fi
 22
 23 echo "##############################################"
 24 echo
```

* Another way to do line 7:

`if [ -f /var/run/httpd/httpd.pid ]`, where `-f DIRECTORY` checks if the directory exists.

### Loops

#### For Loops

```
for var in <list>
do
    <commands>
done
```

* An example:

```
  1 #!/bin/bash
  2
  3 for VAR1 in java .net python ruby php
  4 do
  5    echo "Looping..."
  6    echo "#####################################"
  7    echo "Value of VAR1 is $VAR1"
  8    echo "#####################################"
  9    date
  10 done
```

* Another example for adding users:

```
  1 #!/bin/bash
  2
  3 MYUSERS="alpha beta gamma"
  4
  5 for usr in $MYUSERS
  6 do
  7    echo "Adding user $usr"
  8    useradd $usr
  9    id $usr
 10    echo "#########################################"
 11 done
```